In [1]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
import wikipediaapi
import google.generativeai as genai
import PyPDF2
from langchain_community.document_loaders import WebBaseLoader
import requests
from bs4 import BeautifulSoup
from langchain.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_chroma import Chroma

c:\Users\99013031\Desktop\urlsumm\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# loader = WebBaseLoader("https://en.wikipedia.org/wiki/Artificial_intelligence")
# loader.requests_kwargs = {'verify':False}
# data = loader.load()

wiki_wiki = wikipediaapi.Wikipedia(
    user_agent='Giskard application',
    language='en',
    extract_format=wikipediaapi.ExtractFormat.WIKI
)
p_wiki = wiki_wiki.page('Virat_Kohli')
data = p_wiki.text

In [3]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000 , chunk_overlap = 10)
docs = text_splitter.create_documents([data])

In [4]:
len(docs)

47

In [9]:
docs[1]

Document(page_content='In 2013, Kohli was ranked number one in the ICC rankings for ODI batsmen. In 2015, he achieved the summit of T20I rankings. In 2018, he was ranked top Test batsman, making him the only Indian cricketer to hold the number one spot in all three formats of the game. He is the first player to score 20,000 runs in a decade. In 2020, the International Cricket Council named him the male cricketer of the decade.')

In [5]:
#local models

# llm = Ollama(model = "llama3")
embeddings = OllamaEmbeddings(model= "nomic-embed-text")

In [7]:
#local vectordb

# db = Chroma.from_documents(docs , embeddings , persist_directory="./chroma_db")

In [6]:
# import google.generativeai as genai
# import dotenv

# api_key = dotenv.get_key(".env", key_to_get = "GEMINI_API_KEY")
# genai.configure(api_key=api_key)

# # gen_config = genai.GenerationConfig(
# #     temperature = 0.1
# # )
# model = genai.GenerativeModel("models/gemini-1.5-flash")

In [17]:
from langchain_google_genai import GoogleGenerativeAI

api_key = dotenv.get_key(".env", key_to_get = "GEMINI_API_KEY")
# gen_config = genai.GenerationConfig(temperature=0.4)
llm = GoogleGenerativeAI(model="models/text-bison-001", google_api_key=api_key, temperature=0.4)


In [7]:
for m in genai.list_models():
    if "embedContent" in m.supported_generation_methods:
        print(m)

Model(name='models/embedding-001',
      base_model_id='',
      version='001',
      display_name='Embedding 001',
      description='Obtain a distributed representation of a text.',
      input_token_limit=2048,
      output_token_limit=1,
      supported_generation_methods=['embedContent'],
      temperature=None,
      top_p=None,
      top_k=None)
Model(name='models/text-embedding-004',
      base_model_id='',
      version='004',
      display_name='Text Embedding 004',
      description='Obtain a distributed representation of a text.',
      input_token_limit=2048,
      output_token_limit=1,
      supported_generation_methods=['embedContent'],
      temperature=None,
      top_p=None,
      top_k=None)


In [10]:
# response = model.generate_content("embedding model for gemini")
# response.text

In [11]:
# from langchain_google_genai import GoogleGenerativeAIEmbeddings

# embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=api_key)

# vector = embeddings.embed_query("hello, world!")
# vector[:5]

In [10]:
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(docs, embeddings)

In [11]:
db.save_local("faiss_index")

In [15]:
# db = FAISS.load_local("faiss_index", embeddings ,allow_dangerous_deserialization=True)

In [12]:
retriever = db.as_retriever()

In [13]:
query = "who is the wife of virat kohli"
context = retriever.invoke(query)
context[2]

Document(page_content='Virat Kohli (Hindi pronunciation: [ʋɪˈɾɑːʈ ˈkoːɦli] ; born 5 November 1988) is an Indian international cricketer and the former captain of the Indian national cricket team. He is a right-handed batsman and an occasional medium-fast bowler. He currently represents Royal Challengers Bengaluru in the IPL and Delhi in domestic cricket. Kohli is widely regarded as one of the greatest batsmen of all time. He holds the record as the highest run-scorer in T20I and IPL, ranks third in ODI, and stands as the fourth-highest in international cricket. He also holds the record for scoring the most centuries in ODI cricket and stands second in the list of most international centuries scored. Kohli was a key member of the Indian team that won the 2011 Cricket World Cup, 2013 ICC Champions Trophy, and captained India to win the ICC Test mace three consecutive times in 2017, 2018,  and 2019.')

In [19]:
prompt_template = """
You are a wikipedia assistant. You will be given a question and excerpts from wikipedia. Provide short and clear answers using the excerpts and question.

context:
{context}

question:
{question}

"""

from langchain import PromptTemplate
from langchain.chains import RetrievalQA
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
qa_chain = RetrievalQA.from_llm(llm=llm, retriever=retriever, prompt=prompt)

# Test that everything works
qa_chain.invoke({"query": "what is "})

{'query': 'what is lunch',
 'result': 'Indian Premier League Orange Cap for most runs: 2016'}